In [12]:
import numpy as np
import pandas as pd

# Influencing factors in media coverage

Our hypothesis is that we can observe come connection between the volume of the appearance of a disaster in the media and the characteristics of the disasters. To be concrete we would expect that there are more quotes about disasters with more casualities. In this section our aim is to determine these factors in the observed data of diasaters and examine their contribution to media coverage.

## Reaching the disasters

As we have seen in the previous milestone, we can assume that to a specific disaster the quote realted to it will contain the name of the place and the type of the disaster. So our first goal is to assign the specific quotes according to this assumption to the adequate events.

Step 1: Collect the names and places of the biggest disasters.

In [13]:
disasters = pd.read_csv("datasets/emdat/emdat_public_2021_11_06_clean.csv")
disasters["Event Name"] = disasters["Disaster Type"]+" "+disasters["Country"]
disasters.head()

,Unnamed: 0,Dis No,Disaster Type,Disaster Subtype,Event Name,Country,ISO,Continent,OFDA Response,Appeal,...,No Injured,No Affected,No Homeless,Total Affected,Reconstruction Costs ('000 US$),Insured Damages ('000 US$),Total Damages ('000 US$),human_readable_name,start_date,end_date
0,0,2010-0017-HTI,Earthquake,Ground movement,Earthquake Haiti,Haiti,HTI,Americas,True,NaN,...,300000.0,3400000.0,NaN,3700000.0,11500000.0,200000.0,8000000.0,2010 Earthquake in Haiti,2010-01-01,2010-12-31
1,5,2010-0372-RUS,Extreme temperature,Heat wave,Extreme temperature Russian Federation (the),Russian Federation (the),RUS,Europe,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,400000.0,2010 Extreme temperature in Russian Federatio...,2010-01-01,2010-12-31
2,10,2010-9082-SOM,Drought,Drought,Drought Somalia,Somalia,SOM,Africa,False,Yes,...,NaN,4000000.0,NaN,4000000.0,NaN,NaN,NaN,2010 Drought in Somalia,2010-01-01,2011-12-31
3,11,2011-0082-JPN,Earthquake,Tsunami,Earthquake Japan,Japan,JPN,Asia,True,NaN,...,5933.0,362887.0,NaN,368820.0,NaN,37500000.0,210000000.0,2011 Earthquake in Japan,2011-01-01,2011-12-31
4,16,2015-0144-NPL,Earthquake,Ground movement,Earthquake Nepal,Nepal,NPL,Asia,True,NaN,...,17932.0,5621790.0,NaN,5639722.0,NaN,100000.0,5174000.0,2015 Earthquake in Nepal,2015-01-01,2015-12-31


Step 2: Collect the specific quotes

TODO: We can use these function from the previous milestone, with the words of the event name (splitted).

1. Transfer the models used for classifying disasters on tweets (https://www.kaggle.com/c/nlp-getting-started/overview) to classifying disasters in Quotebank.
Detailed description from the readme: "We would also like to investigate how the models trained on disaster tweets datasets used for classifying natural disaster in the tweets will perform on the quotes from the Quotebank. To do so, we propose to train simple BERT-based sentence encoding with a Logistic Regression Classifier."

2. Classify whether a certain quote is talking about a certain disaster by using simple regexes that would include some information about the place the disaster happened in.
Description from the readme: "In tackling the research question of analyzing how long natural disasters will be talked about, the crucial component is to classify whether a specific quote is talking about a specific disaster. The method we propose to find whether a quote is about a certain disaster is to do simple text searching to match the expected words. These words might include some details about the place where the disaster happened. Depending on the scalability of the method, we would perform the analysis on a subset of the disasters. Assuming that a number of disasters happen in places that usually do not receive much media attention (like Fukushima in world news), we could for example select these disasters and then search for quotes that mention these places, hoping for good precision."

In [14]:
def contain_all_words_regex(words):
    return "".join([f"(?=.*{word})" for word in words])

def quotes_that_contain_all_words(words):
    return dataset[dataset.quotation.str.lower().str.contains(contain_all_words_regex(words), regex=True)]

TODO: Suggestion: here probably we should add a new column to the quotes dataset, where we store either the ID of the disaster or the word pair of the disaster. 

Also we can remove all unneccassary quotes in this section. 

## Determine the volume of media coverage

TODO: In this part I would suggest to determine a volume indicator based on the following. 
- the number of days between the first media appearance and the last (maybe we can leave out the last few quotes if there are some outliers)
- Maybe we can assume that the distribution of the appearances follow a power law.
- The highest number of quotes per day and all of the quotes about the event also can be an important indicator. 

In this point we should assign the these media coverage indicators to the disasters.

## Determine the contibuting factors

Here we should find our dataset about the country indicators and merge it to the disaster dataset. 

We should find first the relevant columns in the disaster dataset which can be tested that it has an effect of the volume numbers. 

## Visualisations

This is the right place to make some nice visualizations for the datastory. 